In [2]:
from pymongo import MongoClient
import requests as rq
import datetime as dt
import time

cardiac = [
    'http://nosql.hpeixoto.me/api/sensor/3001',
    'http://nosql.hpeixoto.me/api/sensor/3002',
    'http://nosql.hpeixoto.me/api/sensor/3003',
    'http://nosql.hpeixoto.me/api/sensor/3004',
    'http://nosql.hpeixoto.me/api/sensor/3005'
]

blood = [
    'http://nosql.hpeixoto.me/api/sensor/4001',
    'http://nosql.hpeixoto.me/api/sensor/4002',
    'http://nosql.hpeixoto.me/api/sensor/4003',
    'http://nosql.hpeixoto.me/api/sensor/4004',
    'http://nosql.hpeixoto.me/api/sensor/4005'
]

url = "mongodb://localhost:27017"
client = MongoClient(url)
db = client.sensors
col = db['sensors']

# Only the first result matters
def getLastTimestamp(sensid):
    m = col.find({"sensorid": sensid},{"_id":0, "timestamp": 1}).sort("_id",-1).limit(1)
    try: 
        return dt.datetime.strptime((m.next())['timestamp'], "%Y-%m-%d %H:%M:%S")
    except:
        return dt.datetime(1,1,1)

# Only insert if the timestamp is more recent than the last on the db
def maybeInsert(data,tsjson):
    tsdb = getLastTimestamp(data['sensorid'])
    if(tsdb >= tsjson):
        print("Sensor is already updated. Skipping the current sensor data.\n")
    else:
        print("Adding sensor data to the DB.\n")
        col.insert_one(data)

# Insert into the collection
def parseJson(data,typ):
    print(">> Currently on sensor: type = {0}, ID = {1}".format(typ, data['sensorid']))
    ts = dt.datetime.strptime(data['timestamp'], "%Y-%m-%d %H:%M:%S")
    maybeInsert(data,ts)

while True:
    print("New Round of GET Requests...")
    for sensor in cardiac:
        data = rq.get(sensor).json()
        parseJson(data, 'cardiac')

    for sensor in blood:
        data = rq.get(sensor).json()
        parseJson(data, 'blood')
    
    time.sleep(5)


New Round of GET Requests...
>> Currently on sensor: type = cardiac, ID = 3001
Adding sensor data to the DB.

>> Currently on sensor: type = cardiac, ID = 3002
Adding sensor data to the DB.

>> Currently on sensor: type = cardiac, ID = 3003
Adding sensor data to the DB.

>> Currently on sensor: type = cardiac, ID = 3004
Adding sensor data to the DB.

>> Currently on sensor: type = cardiac, ID = 3005
Adding sensor data to the DB.

>> Currently on sensor: type = blood, ID = 4001
Adding sensor data to the DB.

>> Currently on sensor: type = blood, ID = 4002
Adding sensor data to the DB.

>> Currently on sensor: type = blood, ID = 4003
Adding sensor data to the DB.

>> Currently on sensor: type = blood, ID = 4004
Adding sensor data to the DB.

>> Currently on sensor: type = blood, ID = 4005
Adding sensor data to the DB.

New Round of GET Requests...
>> Currently on sensor: type = cardiac, ID = 3001
Adding sensor data to the DB.

>> Currently on sensor: type = cardiac, ID = 3002
Adding sens

KeyboardInterrupt: 